In [1]:
"""
this block is needed at the beginning of all notebooks to be able to load modules
"""
import os
import sys
from pathlib import Path
import pandas as pd
import math
import numpy as np
from scipy.signal import detrend
import pywt

# Automatically find and append the project root
root_path = Path.cwd().resolve().parents[1]  # adjust level as needed
data_path = os.path.join(root_path, "data")
src_path = os.path.join(root_path, "src")

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))


In [2]:
from signal_modules.base.signal_module import SignalModule
from regression.Kalman_filter import KalmanFilter

In [3]:
noise = 1e-3
kalman = KalmanFilter(process_noise=noise, obs_noise_scale=3 * noise)

In [4]:
signal_module = SignalModule(oracle=kalman, smoothing_window=1)

In [5]:
initial_time = pd.to_datetime('2023-06-01T12:00:00+00:00')
final_time = pd.Timestamp('2024-01-01T12:00:00+00:00')

tickers = ["SPY", "NVDA", "TSLA", "BRK-B"]  # List of tickers
data_files = [os.path.join(data_path, f"{ticker}.parquet") for ticker in tickers]
data = {
    ticker: pd.read_parquet(path).loc[initial_time:final_time]
    for ticker, path in zip(tickers, data_files)
}
timeline = pd.DatetimeIndex(
    pd.to_datetime(sorted(set().union(*[df.index for df in data.values()])))
)
data = {tk: df.reindex(timeline).ffill() for tk, df in data.items()}
closing_data = {tk: df['adjClose'] for tk, df in data.items()}

In [6]:
signal_module.generate_signals({tk: s[-40:-1] for tk, s in closing_data.items()})

SignalResult(signals=array([ 0.354043  ,  0.37089522, -0.77664682,  0.17972996]), raw_predictions=array([468.28610957,  49.32377249, 255.54033994, 357.18819958]), smoothed_predictions=array([[467.54853792,  49.18561958, 257.35211772, 356.89513267],
       [468.28610957,  49.32377249, 255.54033994, 357.18819958]]), volatility_adjustments=array([1.29162297, 0.76109548, 0.67665944, 1.24555828]), tickers=['SPY', 'NVDA', 'TSLA', 'BRK-B'])

Analyzing shared timeline across tickers...
Found 146 shared trading days
Date range: 2023-06-02 00:00:00+00:00 to 2023-12-29 00:00:00+00:00
Generating historical signals...


Processing timeline: 100%|██████████| 106/106 [00:13<00:00,  7.64it/s]


Successfully generated signals for 106 time points
Creating diagnostic visualizations...


DIAGNOSTIC SUMMARY
Analysis period: 2023-08-01 00:00:00+00:00 to 2023-12-29 00:00:00+00:00
Number of signal observations: 106
Smoothing window used: 1 days

Signal Statistics by Ticker:
----------------------------------------
     SPY: μ=+0.071, σ=0.318, range=[-0.561, +0.712]
    NVDA: μ=+0.049, σ=0.439, range=[-0.695, +0.707]
    TSLA: μ=+0.008, σ=0.450, range=[-0.728, +0.734]
   BRK-B: μ=+0.024, σ=0.315, range=[-0.598, +0.609]

Signal Correlation Matrix:
------------------------------
             SPY    NVDA    TSLA   BRK-B
     SPY   1.000   0.544   0.507   0.474
    NVDA   0.544   1.000   0.403   0.205
    TSLA   0.507   0.403   1.000   0.115
   BRK-B   0.474   0.205   0.115   1.000


In [7]:
signal_module.diagnostics(data=closing_data)